# <span style='color:Blue'> Etape de construction de la Base de Donnée T  </span>

Le but de cette partie est de contruire une base de donnée (utile pour Projet_2.ipynb) de travail nommée "T". 

Les fichiers csv qui interviennent dans la construction de "T" sont : "RAW_recipesnew.csv", "PP_recipes.csv" et "RAW_interactions.csv".

## 1. Importation des librairies

In [71]:
import pandas as pd
import numpy as np
import utiles     # On importe 'utiles.py' qui comporte toutes les fonctions utiles

## 2. Importation de données

### Raw_interactions : Merci d'importer le fichier "RAW_interactions.csv" dans le répertoire afin d'éxecuter ce code.

La base de donnée "RAW_interactions" comporte l'avis des utilisateurs sur les récettes. On y trouve les notes (rating) attribuées à ces recettes. On précise qu'une recette pourrait avoir plusieurs notes différentes. 
Ce qui fait qu'elle comporte 1 132 367 observations. De cette base de donnée, on va juste extraire la variable "rating".

In [72]:
data1 = pd.read_csv("RAW_interactions.csv")
data1.head(2)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."


In [73]:
print("Raw_interactions comporte", data1.shape[0], "observations et", data1.shape[1], "variables.")

Raw_interactions comporte 1132367 observations et 5 variables.


### PP_recipes : Merci d'importer le fichier "PP_recipes.csv" dans le répertoire afin d'exécuter ce code

La base de donnée "PP_recipes" comporte les ingrédients des récettes tokénisés. 
De cette base de donnée, on va juste extraire la variable "calorie_level" pour la construction de notre base de donnée T.

In [74]:
data2 = pd.read_csv("PP_recipes.csv")
data2.head(1)

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"


In [75]:
print("PP_recipes comporte", data2.shape[0], "observations et", data2.shape[1], "variables.")

PP_recipes comporte 178265 observations et 8 variables.


### RAW_recipesnew

La base de donnée "RAW_recipesnew" est extraite de la donnée "RAW_recipes" fournie. "RAW_recipesnew" ne comporte que les récettes dont le temps de cuisson (variable minute) n'est pas nul, ne vaut pas 1 minute et ne depasse pas non plus un MOIS. Contrairement à RAW_recipes, nous avons restreint le nombre de variables dans "RAW_recipesnew". Nous gardons les variables id du récette, id du contributeur, le nom des récettes, l'étape, le nombre d'ingrédients et 7 autres variables qui proviennent du split de la variable nutrition.

In [76]:
data3 = pd.read_csv("RAW_recipesnew.csv")
data3.head(1)

,id,contributor_id,name,minutes,n_steps,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates
0,137739,47892,arriba baked winter squash mexican style,55,11,7,51.5,0.0,13.0,0.0,2.0,0.0,4.0


In [77]:
print("RAW_recipesnew comporte", data3.shape[0], "observations et", data3.shape[1], "variables." )

RAW_recipesnew comporte 229945 observations et 13 variables.


## 3. Rajouter "rating moyen" dans "RAW_recipesnew" pour obtenir "D"

### calcul du rating moyen de chaque récette

Comme une récette pourrait avoir plusieurs rating. On décide donc de rétenir le 'rating moyen' pour chaque récette afin de procéder au merge car dans "RAW_recipesnew", tous les id sont uniques.

In [78]:
rating_mean_by_id = data1.groupby('recipe_id')['rating'].mean()
rating_mean_by_id = rating_mean_by_id.astype('int') # transformer en entier le 'rating moyen'.
rating_mean_by_id.head(3)

recipe_id
38    4
39    3
40    4
Name: rating, dtype: int32

In [79]:
print("Les modalités de rating sont :", sorted(rating_mean_by_id.unique()))

Les modalités de rating sont : [0, 1, 2, 3, 4, 5]


### merge entre "rating moyen" et "RAW_recipesnew"

On inclut le "rating moyen" dans "RAW_recipesnew" en faisant une merge. Le merge s'effectue grâce à l'id des récettes.
Pour une question de simplicité, on nomme le 'rating moyen' calculé par 'rating' tout simplement. On sait que dans cette variable, une récette a une seule note et non plusieurs.

In [80]:
D = pd.merge(rating_mean_by_id, data3, left_on='recipe_id', right_on='id', how='inner')
D.head(1)

,rating,id,contributor_id,name,minutes,n_steps,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates
0,4,38,1533,low fat berry blue frozen dessert,1485,13,4,170.9,3.0,120.0,1.0,6.0,6.0,12.0


## 4. Rajouter "calorie_level" dans "D" créé au dessus

### Extraction de 'calorie_level'

On extrait la variable 'calorie_level' avec l'id des récettes de la base de donnée de PP_recipes (data2) pour l'inclure dans la base de donneée 'D'.

In [81]:
calorie_level = data2[['id','calorie_level']]
calorie_level.head(3)

,id,calorie_level
0,424415,0
1,146223,0
2,312329,1


### Merge entre "calorie_level" et "D" pour obtenir "T"

Le merge s'effectue une fois de plus avec l'id des récettes. On obtient la base de donnée "T".

In [82]:
T = pd.merge(calorie_level, D, left_on='id', right_on='id', how='inner')
T.head(2)

,id,calorie_level,rating,contributor_id,name,minutes,n_steps,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates
0,424415,0,4,496803,aromatic basmati rice rice cooker,61,6,5,228.2,2.0,2.0,8.0,9.0,1.0,15.0
1,146223,0,5,229619,pumpkin pie a la easy,55,10,12,249.4,16.0,92.0,8.0,11.0,27.0,11.0


## 5. Création de 3 nouvelles variables 

On crée 3 nouvelles variables catégorielles nommées 'minutes_discrt', 'n_ingredients_discrt' et 'rating_bin'.
Les deux premières proviennent de la discrétisation des variables continues 'minutes' et 'n_ingredients'.
La dernière 'rating_bin' est une variable binaire qui s'obtient en additionnant les modalités 0, 1, 2 et 3 d'une part et les modalités 4 et 5 d'autre part.

### discretisation de la variable "minutes"

In [83]:
# Discretisation de la variable quantitative "minutes" en 5 classes
quantile_minutes = 5                         #nombre de classe
labels_minutes = ['0', '1', '2', '3', '4']   #nom de classes
minutes_discrt = pd.qcut(T['minutes'], q=quantile_minutes, labels = labels_minutes)
print("Les 5 modalités de la variable 'minutes_discrt' sont :", sorted(minutes_discrt.unique()))

Les 5 modalités de la variable 'minutes_discrt' sont : ['0', '1', '2', '3', '4']


### discretisation de la variable "n_ingredients"

In [84]:
# Discretisation de la variable quantitative "n_ingredients" en 5 classes
quantile_nbreingre = 5
labels_ingredients = ['0', '1', '2', '3', '4']
n_ingredients_discrt = pd.qcut(T['n_ingredients'], q=quantile_nbreingre, labels = labels_ingredients)
print("Les 5 modalités de la variable 'n_ingredients_discrt' sont :", sorted(n_ingredients_discrt.unique()))

Les 5 modalités de la variable 'n_ingredients_discrt' sont : ['0', '1', '2', '3', '4']


### création d'une nouvelle variable "rating_bin"

In [85]:
rating_bin = T['rating'].replace({0:'0', 1:'0', 2:'0', 3:'0', 4:'1', 5:'1'})

### insertion de ces variables dans la base de donnee "T"

Enfin, on inclut ces 3 variables dans la base de donnée "T".

In [86]:
# On inclut les variables discrétisées dans la base de donnee T
T['minutes_discrt'] = minutes_discrt
T['n_ingredients_discrt'] = n_ingredients_discrt
T['rating_bin'] = rating_bin

In [93]:
# Convertir en entier les variables ci-dessous

T['total_fat'] = T['total_fat'].astype(int)
T['sugar'] = T['sugar'].astype(int)
T['sodium'] = T['sodium'].astype(int)
T['protein'] = T['protein'].astype(int)
T['total_fat'] = T['total_fat'].astype(int)
T['saturated_fat'] = T['saturated_fat'].astype(int)
T['carbohydrates'] = T['carbohydrates'].astype(int)
T['n_steps'] = T['n_steps'].astype(int)
T['minutes_discrt'] = T['minutes_discrt'].astype(int)

## Base de donnée T

In [88]:
# On impose un ordre d'affichage des variables
T = pd.DataFrame.reindex(T, columns = ['id', 'contributor_id', 'name', 'minutes', 'minutes_discrt',
                                       'n_steps', 'n_ingredients', 'n_ingredients_discrt', 
                                       'calories', 'calorie_level', 'total_fat', 'saturated_fat', 'sugar', 'sodium',
                                       'protein', 'carbohydrates', 'rating', 'rating_bin'])

In [89]:
T.head(2)

,id,contributor_id,name,minutes,minutes_discrt,n_steps,n_ingredients,n_ingredients_discrt,calories,calorie_level,total_fat,saturated_fat,sugar,sodium,protein,carbohydrates,rating,rating_bin
0,424415,496803,aromatic basmati rice rice cooker,61,3,6,5,0,228.2,0,2,1,2,8,9,15,4,1
1,146223,229619,pumpkin pie a la easy,55,3,10,12,3,249.4,0,16,27,92,8,11,11,5,1


In [90]:
print("La base de donnée T comporte", T.shape[0], "observations et", T.shape[1], "variables." )

La base de donnée T comporte 177224 observations et 18 variables.


In [94]:
print("La base de donnée T comporte", T.isnull().sum().sum(), "donnée manquante.")

La base de donnée T comporte 0 donnée manquante.


### Exporter la base de donnée T dans un fichier "csv"

On exporte la base de donnée T ainsi créée dans un fichier csv pour pouvoir l'utiliser dans notre projet 2.

In [95]:
T.to_csv("T.csv")